##### Silver Speech and Golden Silence: Spoiler Detection Project

### Review-wise Balanced Random Forest with Metadata (combined in 1 feature)

In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier

#### Load Data

In [2]:
#Load train data
train = pd.read_json('data/train_reduced.json')
train.drop(['user_id', 'title'], inplace = True, axis = 1)
train.reset_index(drop = True, inplace = True)

In [3]:
#Load validation data
val = pd.read_json('data/validation_reduced.json')
val.drop(['user_id', 'title'], inplace = True, axis = 1)
val.reset_index(drop = True, inplace = True)

In [4]:
#Load reviews as a whole for train and validation
X_tr = pd.read_csv('data/X_train.csv')
X_v = pd.read_csv('data/X_val.csv')
#Rename columns
X_tr.rename(columns = {"0": 'review_whole'}, inplace = True)
X_v.rename(columns = {"0": 'review_whole'}, inplace = True)
#Drop unnamed columns
X_tr.drop('Unnamed: 0', inplace = True, axis = 1)
X_v.drop('Unnamed: 0', inplace = True, axis = 1)
#Reset indices
X_tr.reset_index(drop = True, inplace = True)
X_v.reset_index(drop = True, inplace = True)

In [5]:
#Combine dataframes and reviews 
train_df = pd.concat([train, X_tr], axis = 1)
val_df = pd.concat([val, X_v], axis = 1)

In [6]:
#Save data for later use
train_df.to_json('data/train_only_needed_columns.json')
val_df.to_json('data/val_only_needed_columns.json')

In [7]:
#Since the data is imbalanced, we downsample the non-spoilers. This further reduces the sample to about 25,000.
def downsample_nonspoilers_reviewwise(df):
    df_majority = df[df['spoiler_dum'] == 0] #nonspoilers
    df_minority = df[df['spoiler_dum'] == 1] #spoilers
    N = 2*len(df_minority)
    
    # Downsample majority labels equals three times the number of samples in the minority class while keeping the original genre proportion
    df_majority_ = df_majority.groupby('genre', group_keys = False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df))))).sample(frac = 1)

    # Concatenate the majority and minority dataframes
    sample = pd.concat([df_majority_, df_minority], axis = 0)
    
    sample.reset_index(inplace = True, drop = True)
    return sample

In [8]:
train_df = downsample_nonspoilers_reviewwise(train_df)
train_df.shape

(180169, 6)

In [9]:
#New column with genre and review text
train_df[['text_and_genre']] = train_df.genre + ' ' + train_df.review_whole
val_df[['text_and_genre']] = val_df.genre + ' ' + val_df.review_whole

In [10]:
X_train = train_df.drop('spoiler_dum', axis = 1)
y_train = train_df.spoiler_dum

In [11]:
X_val = val_df.drop('spoiler_dum', axis = 1)
y_val = val_df.spoiler_dum

In [12]:
X_train_rev = X_train.text_and_genre
X_val_rev = X_val.text_and_genre

#### Pipeline for Balanced Random Forest with Review Texts and Genre Metadata

In [13]:
#Function for model training and prediction
def run_model(pipeline, X_train, y_train, X_test, y_test):
    #Fit the model
    model = pipeline.fit(X_train, y_train)
    
    #Predict labels of test data
    y_pred = model.predict(X_test)
    
    #Print classification report and confusion matrix
    return print(classification_report(y_test, y_pred)), print(confusion_matrix(y_test, y_pred, normalize = 'true'))
    return y_pred

In [14]:
#Build a pipeline for feature extraction with TF IDF and SGD
#TFIDF
tfidf = TfidfVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 100)

#Alternative: CountVectorizer
vectorizer = CountVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 100)

#Define TomekLinks for Undersampling
#tl = TomekLinks(sampling_strategy = 'majority', n_jobs = -1)

#Feature Selection: K-Best
feat = SelectKBest(score_func = chi2, k = 10000)

#SGD
sgd = SGDClassifier(random_state = 42, penalty = 'l2', shuffle = True, n_jobs = -1, max_iter = 1000, 
                                       loss = 'hinge', class_weight = 'balanced', alpha = .0001)

#Balanced Random Forest classifier
brf = BalancedRandomForestClassifier(n_estimators = 100, random_state = 42, verbose = 5, n_jobs = -1, class_weight = 'balanced', oob_score = True)


#CalibratedClassifier
cal_sgd = CalibratedClassifierCV(base_estimator = brf)

pipe = Pipeline([('tfidf', tfidf),('select_kbest', feat), ('cal_classifier', cal_sgd)])

In [15]:
#run the model review-wise (validation)
run_model(pipe, X_train_rev.astype('str'), y_train, X_val_rev.astype('str'), y_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 100
building tree 3 of 100building tree 4 of 100

building tree 1 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.4min


building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100building tree 26 of 100

building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 32 of 100
building tree 31 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.4min


...

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  8.9min


building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 13.8min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   11.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   18.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   12.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   18.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 ta

              precision    recall  f1-score   support

           0       0.99      0.81      0.89    235341
           1       0.26      0.86      0.40     18062

    accuracy                           0.82    253403
   macro avg       0.63      0.84      0.65    253403
weighted avg       0.94      0.82      0.86    253403

[[0.81402306 0.18597694]
 [0.13580999 0.86419001]]


(None, None)